This notebook is part of https://github.com/AudioSceneDescriptionFormat/splines, see also https://splines.readthedocs.io/.

# Spherical Linear Interpolation (Slerp)

"Great arc in-betweening"

The term "Slerp" for "**s**pherical **l**inear int**erp**olation"
has been coined by
<cite data-cite="shoemake1985animating">Shoemake (1985)</cite>
(section 3.3).
It is defined as:

\begin{equation*}
\operatorname{Slerp}(q_1, q_2; u) =
q_1 \left(q_1^{-1} q_2\right)^u
\end{equation*}

The parameter $u$ moves from $0$ (where the expression simplifies to $q_1$)
to $1$ (where the expression simplifies to $q_2$).

The [Wikipedia article for Slerp](https://en.wikipedia.org/wiki/Slerp#Quaternion_Slerp)
provides four equivalent ways to describe the same thing:

\begin{align*}
\operatorname{Slerp}(q_0, q_1; t)
& = q_0 \left(q_0^{-1} q_1\right)^t \\
& = q_1 \left(q_1^{-1} q_0\right)^{1-t} \\
& = \left(q_0 q_1^{-1}\right)^{1-t} q_1 \\
& = \left(q_1 q_0^{-1}\right)^t q_0
\end{align*}

Shoemake also provides an alternative formulation:

\begin{equation*}
\operatorname{Slerp}(q_1, q_2; u) =
\frac{\sin (1-u) \theta}{\sin \theta} q_1 +
\frac{\sin u \theta}{\sin \theta} q_2,
\end{equation*}

where the dot product
$q_1 \cdot q_2 = \cos \theta$.

In [ ]:
import numpy as np

[helper.py](helper.py)

In [ ]:
from helper import angles2quat, animate_rotations, display_animation

In [ ]:
def slerp(one, two, t):
    return (two * one.inverse())**t * one

In [ ]:
q1 = angles2quat(45, -20, -60)
q2 = angles2quat(-45, 20, 30)

In [ ]:
times = np.linspace(0, 1, 50)

In [ ]:
ani = animate_rotations({
    'slerp(q1, q2)': slerp(q1, q2, times),
    'slerp(q1, -q2)': slerp(q1, -q2, times),
}, figsize=(6, 3))

In [ ]:
display_animation(ani, default_mode='reflect')

## Piecewise Slerp

In [ ]:
from splines.quaternion import PiecewiseSlerp

In [ ]:
s = PiecewiseSlerp([
    angles2quat(0, 0, 0),
    angles2quat(90, 0, 0),
    angles2quat(90, 90, 0),
    angles2quat(90, 90, 90),
], grid=[0, 1, 2, 3, 6], closed=True)

In [ ]:
times = np.linspace(s.grid[0], s.grid[-1], 100)

In [ ]:
ani = animate_rotations({
    'Piecewise Slerp': s.evaluate(times),
}, figsize=(3, 3))

In [ ]:
display_animation(ani, default_mode='loop')

## Slerp vs. Nlerp

"**n**ormalized **l**inear int**erp**olation"

In [ ]:
from splines.quaternion import Quaternion

In [ ]:
def lerp(one, two, t):
    """Linear interpolation.
    
    t can go from 0 to 1.
    
    """
    return (1 - t) * one + t * two

In [ ]:
def nlerp(one, two, t):
    """Normalized linear interpolation.
    
    Linear interpolation in 4D quaternion space,
    normalizing the result.
    
    t can go from 0 to 1.
    
    """
    one = np.array(one.xyzw)
    two = np.array(two.xyzw)
    *vector, scalar = lerp(one, two, t)
    return Quaternion(scalar, vector).normalize()

In [ ]:
q1 = angles2quat(-60, 10, -10)
q2 = angles2quat(80, -35, -110)

In [ ]:
assert q1.dot(q2) > 0

In [ ]:
ani_times = np.linspace(0, 1, 50)

In [ ]:
ani = animate_rotations({
    'Slerp': slerp(q1, q2, ani_times),
    'Nlerp': [nlerp(q1, q2, t) for t in ani_times],
}, figsize=(6, 3))

In [ ]:
display_animation(ani, default_mode='reflect')

Let's create some still images:

In [ ]:
from helper import plot_rotations

In [ ]:
plot_times = [0, 0.25, 0.5, 0.75, 1]

In [ ]:
plot_rotations({
    'Slerp': slerp(q1, q2, plot_times),
    'Nlerp': [nlerp(q1, q2, t) for t in plot_times],
}, figsize=(6, 3))

Start and end are (by definition) the same,
the middle is also the same (due to symmetry).
And in between, there are very slight differences.

Since the differences are barely visible, we can try a more extreme example:

In [ ]:
q3 = angles2quat(-170, 0, 45)
q4 = angles2quat(120, -90, -45)

In [ ]:
assert q3.dot(q4) < 0

Please note that this is a rotation
by an angle of far more than 180 degrees!

In [ ]:
ani = animate_rotations({
    'Slerp': slerp(q3, q4, ani_times),
    'Nlerp': [nlerp(q3, q4, t) for t in ani_times],
}, figsize=(6, 3))

In [ ]:
display_animation(ani, default_mode='reflect')

In [ ]:
plot_rotations({
    'Slerp': slerp(q3, q4, plot_times),
    'Nlerp': [nlerp(q3, q4, t) for t in plot_times],
}, figsize=(6, 3))